In [1]:
### TODO:
# * add train/valid/test generators to data
# * need option to apply preprocessor when requesting frame data
# * make sure don't recompute sequences if inputs don't change

In [2]:
# whether to log each feature and sequence status
verbose = 1

In [3]:
import os
import pandas as pd
import numpy as np
import json
from PIL import Image
import cv2
from sklearn.utils import shuffle
import sys
sys.path.append('..')

import h5py

In [4]:
# import pretrained model functions
from deepvideoclassification.models import precompute_CNN_features
from deepvideoclassification.models import load_pretrained_model_preprocessor
from deepvideoclassification.models import load_pretrained_model

# import pretrained model properties
from deepvideoclassification.models import pretrained_model_len_features
from deepvideoclassification.models import pretrained_model_sizes
from deepvideoclassification.models import pretrained_model_names, poolings

Using TensorFlow backend.


In [5]:
# setup paths
pwd = os.getcwd().replace("deepvideoclassification","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [6]:
# setup logging
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
logger = logging.getLogger()

In [7]:
# read vid folders
def get_video_paths():
    """
    Return list of video paths 

    Videos should be in /data/video_1/, /data/video_2/ style folders 
    with sequentially numbered frame images e.g. /data/video_1/frame00001.jpg

    There should be at least 3 videos, 1 for each of train/test/valid splits
    Split assignment is given in /data/labels.csv (also to be provided by user)

    Functionality to use different parts of a video as train/valid/test 
    not currently implemented.
    """
    path_videos = []
    for filename in os.listdir(path_data):
        if os.path.isdir(os.path.join(path_data, filename)):
            path_videos.append(filename)

    path_videos = [path_data + v + '/' for v in path_videos]

    # make sure that there is video data in /data/ and give instructions if not done correctly
    assert len(path_videos)>0, "There need to be at least 3 video folders (at least 1 for each of train, valid, \
    and test splits) in /data/ - each video should be its own folder of frame images with ascending time-ordered \
    filenames e.g. /data/vid1/frame00001.jpg ... videos assignment to train/valid/test split should be given in \
    /data/labels.csv ... cross-validation or train/valid/test splits within a single long video not currently implemented"

    return path_videos

In [8]:
def resize_frames(target_size):
    """
    Resize the frames of all videos and save them to /cache/ 
    to make model fitting faster .

    We resize once upfront rather than each we use a pretrained model or architecture.

    Our models require inputs resized to:
    * 224 x 224 VGG16, ResNet50, DenseNet, MobileNet
    * 299 x 299 XCeption, InceptionV3, InceptionResNetV2
    * 112 x 112 3D CNN 
    """

    if not os.path.exists(path_cache + 'frames/' + str(target_size[0]) + "_" + str(target_size[1]) + '/'):
        
        os.makedirs(path_cache + 'frames/' + str(target_size[0]) + "_" + str(target_size[1]) + '/')

        # read vid paths
        path_videos = get_video_paths()

        # loop over all vids and resize frames, saving to new folder in /cache/frames/
        for c,path_video in enumerate(path_videos):

            logger.info("resizing vid {}/{} to {}x{}".format(c+1,len(path_videos),target_size[0], target_size[1]))

            # get vid name from path
            video_name = path_video.split("/")[-2]

            # create directory for resized frames - just storing arrays now so commented out
            # e.g. path_vid_resized = /cache/frames/224_224/s23-4847/
            # path_vid_resized = path_cache + 'frames/'
            # path_vid_resized += str(target_size[0]) + "_" + str(target_size[1]) + '/' 
            # path_vid_resized += video_name + '/'

            # load frame paths for vid
            path_frames = os.listdir(path_video)
            path_frames = [path_video + f for f in path_frames if f != '.DS_Store']
            path_frames.sort()

            # load frames
            frames = []
            for path_frame in path_frames:

                # open image and resize
                filename = path_frame.split("/").pop()
                img_frame = Image.open(path_frame)
                img_frame = img_frame.resize(target_size)
                # img_frame.save(path_vid_resized + filename, "JPEG", quality = 100)

                # convert to array and append to list
                img_frame = np.array(img_frame)
                frames.append(img_frame)

            # save array of resized frames
            np.save(path_cache + "frames/" + str(target_size[0]) + "_" + str(target_size[1]) + "/" + video_name, np.array(frames))

In [9]:
def get_labels():
    # read labels - should be CSV with columns "video","frame","label","split"
    # e.g. "s1-218", "s1-218-00001.jpeg", "noseal", "train"
    labels = None
    try:
        labels = pd.read_csv(path_data + 'labels.csv', usecols=['video','frame','label','split'])
    except ValueError as e:
        raise Exception("Labels file must contain columns ['video','frame','label','split'] - if you only have ['video','frame','label'], use the helper function add_splits_to_labels_file to add train/valid/test splits to your labels file")
    except FileNotFoundError as e:
        raise Exception("No labels found - please save labels file to /data/labels.csv") from e

    return labels.sort_values(["video","frame"])

In [10]:
def create_video_label_arrays():
    """
    Create numpy array with labels for each vid and a label_map.json file
    in /cache/labels/
    """

    # create folder for labels
    if not os.path.exists(path_cache + 'labels/'):
        os.makedirs(path_cache + 'labels/')

    # load labels
    labels = get_labels()

    # build label_map
    label_dummies = pd.get_dummies(labels, columns = ['label'])

    # get label columns list and build label map dict
    label_columns = []
    label_map = {}
    label_map_idx = 0
    for i, col in enumerate(label_dummies.columns):
        if col[:6] == 'label_':
            label_columns.append(col)
            label_map[label_map_idx] = col
            label_map_idx+=1

    # save label map to json
    with open(path_cache + 'labels/label_map.json', 'w') as fp:
        json.dump(label_map, fp)

    # get video paths
    path_videos = get_video_paths()

    # save numpy array of labels for each vid
    for path_video in path_videos:

        # get vid name from path
        video_name = path_video.split("/")[-2]

        vid_labels = np.array(label_dummies[label_dummies['video'] == video_name][label_columns])

        # save labels array for this vid
        np.save(path_cache + "/labels/" + video_name, np.array(vid_labels))

In [11]:
def load_label_map():
    """
    Returns label map - read from disk
    """

    # load label map from disk
    label_map = None
    try:
        if os.path.exists(path_cache + 'labels/label_map.json'):
            with open(path_cache + 'labels/label_map.json', 'r') as fp:
                label_map = json.load(fp)
        else:
            # build labels and label map
            create_video_label_arrays()
            if os.path.exists(path_cache + 'labels/label_map.json'):
                with open(path_cache + 'labels/label_map.json', 'r') as fp:
                    label_map = json.load(fp)
    except Exception as e:
        logger.error ('label map not found - make sure /data/labels.csv exists and data cache has been built')

    return label_map

In [12]:
class Data(object):
    
    def __init__(self, sequence_length, 
                    return_CNN_features = False, pretrained_model_name = None, pooling = None, 
                    frame_size = None, augmentation = False, oversampling = False,
                    model_weights_path = None, custom_model_name = None):
        """
        Data object constructor
        
        
        :sequence_length: number of frames in sequence to be returned by Data object
        :return_CNN_features: whether to return precomputed features or return frames (or sequences of features/frames if sequence_length>1)

        :return_features: if True then return features (or sequences of feature) from pretrained model, if False then return frames (or sequences of frames)        
        :pretrained_model_name: name of pretrained model (or None if not using pretrained model e.g. for 3D-CNN)
        :pooling: name of pooling variant (or None if not using pretrained model e.g. for 3D-CNN)
        :frame_size: size that frames are resized to (this is looked up for pretrained models)
        :augmentation: whether to apply data augmentation (horizontal flips)
        :oversampling: whether to apply oversampling to create class balance
        
        :model_weights_path: path to custom model weights if we want to load CNN model we've fine-tuned to produce features (e.g. for LRCNN)
        :custom_model_name: custom output name to append to pretrained model name
        
        
        Notes: 
        * if pretrained_model_name != None and return_CNN_features=False then will first apply preprocessor to frames (or frame sequences)
        """
    
        # required params
        self.sequence_length = sequence_length
        self.frame_size = frame_size
        
        # optional params
        self.pretrained_model_name = pretrained_model_name
        self.pooling = pooling
        self.return_CNN_features = return_CNN_features
        self.augmentation = augmentation
        self.oversampling = oversampling
        
        # init model data
        self.x_train = []
        self.y_train = []
        #
        self.x_valid = []
        self.y_valid = []
        # 
        self.x_test = []
        self.y_test = []
        
        # fix case sensitivity
        if type(self.pretrained_model_name) == str:
            self.pretrained_model_name = self.pretrained_model_name.lower()
        if type(self.pooling) == str:
            self.pooling = self.pooling.lower()
        
        ################
        ### Prepare data
        ################
        
        # get video paths
        self.path_videos = get_video_paths()
        
        # create label array for each video and load label map
        create_video_label_arrays()
        self.label_map = load_label_map()
        
        # get labels
        self.labels = get_labels()
        
        # create dict mapping video to train/valid/test split assignment
        video_splits = self.labels[['video','split']].drop_duplicates()
        video_splits.set_index("video", inplace=True)
        video_splits = video_splits.to_dict()['split']
        self.video_splits = video_splits
        
        # look up target size for pretrained model
        if pretrained_model_name is not None:
            self.frame_size = pretrained_model_sizes[pretrained_model_name]
        
        # precompute resized frames (won't recompute if already resized)
        resize_frames(self.frame_size)

        # pre compute CNN features (won't recompute if already computed)
        if return_CNN_features and pretrained_model_name is not None:
            if model_weights_path is not None and custom_model_name is not None:
                # precompute with custom weights input and name
                precompute_CNN_features(self.pretrained_model_name, self.pooling, self.model_weights_path, self.custom_model_name)
            else:
                precompute_CNN_features(self.pretrained_model_name, self.pooling)
            
            
            
        # get preprocessor given pretrained if we will need to apply preprocessor 
        # (i.e. if return_CNN_features == False and pretrained_model_name != None)
        if not return_CNN_features and pretrained_model_name is not None:
            self.preprocess_input = load_pretrained_model_preprocessor(self.pretrained_model_name)
        
        
        ###################################
        ### load features / build sequences
        ###################################
        
        
        # load features/frames from all videos and concat into big array for each of train, valid and test
        if self.sequence_length > 1:
            
            ### sequences
            
            if return_CNN_features:
                
                #####################
                ### feature sequences
                #####################
                
                path_features = path_cache + 'features/' + pretrained_model_name + "/" + pooling + '/'
                if not return_CNN_features and pretrained_model_name is not None:
                    path_features = path_cache + 'features/' + pretrained_model_name + "__" + custom_model_name + "/" + pooling + '/'
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    
                    ### create sequence: features
                    # load precomputed features
                    features = np.load(path_features + video_name + '.npy')
                    # build sequences
                    x = []
                    for i in range(sequence_length, len(features) + 1):
                        x.append(features[i-sequence_length:i])
                    x = np.array(x)
                    
                    
                    ### create sequence: labels
                    # load precomputed labels
                    labels = np.load(path_labels + video_name + '.npy')     

                    # temp lists to store sequences
                    y = []
                    for i in range(sequence_length, len(labels) + 1):
                        y.append(labels[i-1])
                    y = (np.array(y))

                    
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
                        
            else:

                ###################
                ### frame sequences
                ###################
                
                # load resized numpy array
                path_vid_resized = path_cache + 'frames/'
                path_vid_resized += str(self.frame_size[0]) + "_" + str(self.frame_size[1]) + '/' 
                
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    ### create sequence: features
                    # load precomputed frames
                    frames = np.load(path_vid_resized  + video_name + '.npy')
                    
                    # first apply preprocessing if pretrained model given
                    if pretrained_model_name != None:
                        frames = self.preprocess_input(frames)
                    
                    # build sequences
                    x = []
                    for i in range(sequence_length, len(frames) + 1):
                        x.append(frames[i-sequence_length:i])
                    x = np.array(x)
                    
                    
                    ### create sequence: labels
                    # load precomputed labels
                    labels = np.load(path_labels + video_name + '.npy')     

                    # temp lists to store sequences
                    y = []
                    for i in range(sequence_length, len(labels) + 1):
                        y.append(labels[i-1])
                    y = (np.array(y))

                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
                
        else:

            ### not sequence
            
            if return_CNN_features:
                
                ###################
                ### feature vectors
                ###################
                
                path_features = path_cache + 'features/' + pretrained_model_name + "/" + pooling + '/'
                if not return_CNN_features and pretrained_model_name is not None:
                    path_features = path_cache + 'features/' + pretrained_model_name + "__" + custom_model_name + "/" + pooling + '/'
                
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    ### load precomputed features
                    x = np.load(path_features + video_name + '.npy')
                    y = np.load(path_labels + video_name + '.npy')
                    
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
            else:
                
                #################
                ### single frames
                #################
                
                # load resized numpy array
                path_vid_resized = path_cache + 'frames/'
                path_vid_resized += str(self.frame_size[0]) + "_" + str(self.frame_size[1]) + '/' 
                
                path_labels = path_cache + 'labels/'
                
                # read vid paths
                path_videos = get_video_paths()

                # loop over all vids and resize frames, saving to new folder in /cache/frames/
                for c, path_video in enumerate(path_videos):

                    # get vid name from path
                    video_name = path_video.split("/")[-2]
                    
                    # load precomputed numpy arrays for frames and labels
                    x = np.load(path_vid_resized  + video_name + '.npy')
                    y = np.load(path_labels + video_name + '.npy')
                    
                    # apply preprocessing if pretrained model given
                    if pretrained_model_name != None:
                        x = self.preprocess_input(x)
                
                    ### build output
                    if self.video_splits[video_name] == "train":
                        self.x_train.append(x)
                        self.y_train.append(y)
                    if self.video_splits[video_name] == "valid":
                        self.x_valid.append(x)
                        self.y_valid.append(y)
                    if self.video_splits[video_name] == "test":
                        self.x_test.append(x)
                        self.y_test.append(y)
            
            
            
        ########################
        ### reshape list outputs
        ########################
        ## e.g. (9846, 224, 224, 3) for frames [return_CNN_features=True]
        ## or  (9846, 512) for features [return_CNN_features=False]
        self.x_train = np.concatenate(self.x_train, axis=0)
        self.y_train = np.concatenate(self.y_train, axis=0)
        self.x_valid = np.concatenate(self.x_valid, axis=0)
        self.y_valid = np.concatenate(self.y_valid, axis=0)
        self.x_test = np.concatenate(self.x_test, axis=0)
        self.y_test = np.concatenate(self.y_test, axis=0)
        
        
        
        #################################
        ### get file paths for each split
        #################################
        #
        # Note: only makes sense for sequence_length = 1
        
        # get file paths: train
        dflab = self.labels[self.labels['split'] == 'train']
        self.paths_train = list(path_data + dflab['video'] + "/" + dflab['frame'])

        # get file paths: valid
        dflab = self.labels[self.labels['split'] == 'valid']
        self.paths_valid = list(path_data + dflab['video'] + "/" + dflab['frame'])

        # get file paths: test
        dflab = self.labels[self.labels['split'] == 'test']
        self.paths_test = list(path_data + dflab['video'] + "/" + dflab['frame'])
        
        # pull number of classes from labels shape
        self.num_classes = self.y_train.shape[1]
            

    def __str__(self):
        return "x_train: {}, y_train: {} ... x_valid: {}, y_valid: {} ... x_test: {}, y_test: {}".format(self.x_train.shape,self.y_train.shape,self.x_valid.shape,self.y_valid.shape,self.x_test.shape,self.y_test.shape)
            
    def shuffle(self):
        """
        randomize the order of samples in train and valid splits
        """
        ###########
        ### shuffle
        ###########
        # paths will no longer be correct (they're for debugging anyway)
        self.x_train, self.y_train, self.paths_train = shuffle(self.x_train, self.y_train, self.paths_train)
        self.x_valid, self.y_valid, self.paths_valid = shuffle(self.x_valid, self.y_valid, self.paths_valid)

In [13]:
# Even at small sequence lengths, loading the full dataset as 
# a sequence into memory is not feasible so we need to use generators
# that iterate over the dataset without loading it all into memory
# 
# For now, we will assume that we will load the features datasets into memory
# because this is more feasible but for large datasets, we'd want to use generators
# for that too. An implementation for that can be done by pattern matching the implementation below.
# 
# Our frames are in separate directories so we cannot use keras.preprocessing.image.ImageDataGenerator

import keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

# TEST GENERATOR

In [14]:
from deepvideoclassification.models import Architecture

In [21]:
from deepvideoclassification.models import train

In [49]:
pretrained_model_name = "vgg16"
pooling="max"
sequence_length = 5

layer_1_size = 128
layer_2_size = 64
layer_3_size = 32
dropout=0.20

In [95]:
data = Data(sequence_length = sequence_length, 
            return_CNN_features = False, 
            pretrained_model_name=pretrained_model_name,
            pooling = pooling)

num_classes = data.num_classes 
frame_size = data.frame_size
num_features = pretrained_model_len_features[pretrained_model_name]

In [17]:
architecture = Architecture(architecture="image_MLP_trainable", 
                     sequence_length = 1,
                     num_classes = data.num_classes, 
                     frame_size = data.frame_size, 
                     pretrained_model_name='vgg16', 
                     pooling='max',
                     layer_1_size=128, 
                     layer_2_size=32, 
                     layer_3_size=16, 
                     dropout=0.2)

## build generator dataset

In [42]:
!tree /mnt/seals/cache/ -d

/mnt/seals/cache/
├── features
│   ├── inception_resnet_v2
│   │   ├── avg
│   │   └── max
│   ├── inception_v3
│   │   ├── avg
│   │   └── max
│   ├── mobilenetv2_1.00_224
│   │   ├── avg
│   │   └── max
│   ├── resnet50
│   │   ├── avg
│   │   └── max
│   ├── vgg16
│   │   ├── avg
│   │   └── max
│   └── xception
│       ├── avg
│       └── max
├── frames
│   ├── 112_112
│   ├── 224_224
│   ├── 299_299
│   └── 80_80
└── labels

25 directories


## first calc total size

In [96]:
video_splits = data.video_splits

In [124]:
# total number of rows of sequence data we have for each split
# this is not the same as the number of frames since we exclude
# the first (sequence_length-1) frames
total_rows_train = 0
total_rows_valid = 0
total_rows_test = 0

# load resized numpy array
path_vid_resized = path_cache + 'frames/'
path_vid_resized += str(frame_size[0]) + "_" + str(frame_size[1]) + '/' 

path_labels = path_cache + 'labels/'

# read vid paths
path_videos = get_video_paths()

# loop over all vids and resize frames, saving to new folder in /cache/frames/
for c, path_video in enumerate(path_videos):

    # get vid name from path
    video_name = path_video.split("/")[-2]

    # load resized frames
    frames = np.load(path_vid_resized  + video_name + '.npy')

    # build sequences
    x = []
    for i in range(sequence_length, len(frames) + 1):
        x.append(frames[i-sequence_length:i])
    x = np.array(x)

    if video_splits[video_name] == "train":
        total_rows_train += len(x)
    if video_splits[video_name] == "valid":
        total_rows_valid += len(x)
    if video_splits[video_name] == "test":
        total_rows_test += len(x)

In [125]:
# calc shapes required for full sequence dataset
h5_shape_train_x = (total_rows_train, sequence_length, frame_size[0], frame_size[1], 3)
h5_shape_train_y = (total_rows_train, num_classes)

h5_shape_valid_x = (total_rows_valid, sequence_length, frame_size[0], frame_size[1], 3)
h5_shape_valid_y = (total_rows_valid, num_classes)

h5_shape_test_x = (total_rows_test, sequence_length, frame_size[0], frame_size[1], 3)
h5_shape_test_y = (total_rows_test, num_classes)

In [126]:
print("train {} ... {}".format(h5_shape_train_x, h5_shape_train_y))
print("valid {} ... {}".format(h5_shape_valid_x, h5_shape_valid_y))
print("test {} ... {}".format(h5_shape_test_x, h5_shape_test_y))

train (10619, 5, 224, 224, 3) ... (10619, 2)
valid (1360, 5, 224, 224, 3) ... (1360, 2)
test (295, 5, 224, 224, 3) ... (295, 2)


In [127]:
# open h5 file to store big sequence dataset feature and label arrays
# path_h5file = RESIZE -> MODEL -> SEQUENCE LENGTH
f_train = h5py.File('train_sequences' + str(sequence_length) + '.h5', 'a')
f_valid = h5py.File('valid_sequences' + str(sequence_length) + '.h5', 'a')
f_test = h5py.File('test_sequences' + str(sequence_length) + '.h5', 'a')

In [128]:
# initialize h5 datasets
h5_train_x = f_train.create_dataset('sequences', shape= h5_shape_train_x, dtype='uint8')
h5_train_y = f_train.create_dataset('labels', shape= h5_shape_train_y, dtype='uint8')

h5_valid_x = f_valid.create_dataset('sequences', shape= h5_shape_valid_x, dtype='uint8')
h5_valid_y = f_valid.create_dataset('labels', shape= h5_shape_valid_y, dtype='uint8')

h5_test_x = f_test.create_dataset('sequences', shape= h5_shape_test_x, dtype='uint8')
h5_test_y = f_test.create_dataset('labels', shape= h5_shape_test_y, dtype='uint8')

In [129]:
# load resized numpy array
path_vid_resized = path_cache + 'frames/'
path_vid_resized += str(frame_size[0]) + "_" + str(frame_size[1]) + '/' 

path_labels = path_cache + 'labels/'

# read vid paths
path_videos = get_video_paths()

# keep track of where we are in the h5 file
h5_cursor = 0

# loop over all vids and resize frames, saving to new folder in /cache/frames/
for c, path_video in enumerate(path_videos):

    # get vid name from path
    video_name = path_video.split("/")[-2]
    
    print(c, len(path_videos), video_name)

    ### create sequence: features
    # load precomputed frames
    frames = np.load(path_vid_resized  + video_name + '.npy')

    # first apply preprocessing if pretrained model given
#     if pretrained_model_name != None:
#         frames = self.preprocess_input(frames)

    # build sequences
    x = []
    for i in range(sequence_length, len(frames) + 1):
        x.append(frames[i-sequence_length:i])
    x = np.array(x)

    ### create sequence: labels
    # load precomputed labels
    labels = np.load(path_labels + video_name + '.npy')     

    # temp lists to store sequences
    y = []
    for i in range(sequence_length, len(labels) + 1):
        y.append(labels[i-1])
    y = (np.array(y))

    print("XX", h5_cursor, y.shape, "..", x.shape)
    
    ### write this vid's data to relevant h5 dataset
    if video_splits[video_name] == "train":
        h5_train_x[h5_cursor:h5_cursor + x.shape[0], :, :, :, :] = x
        h5_train_y[h5_cursor:h5_cursor + y.shape[0], :] = y
    if video_splits[video_name] == "valid":
        h5_valid_x[h5_cursor:h5_cursor + x.shape[0], :, :, :, :] = x
        h5_valid_y[h5_cursor:h5_cursor + y.shape[0], :] = y
    if video_splits[video_name] == "test":
        h5_test_x[h5_cursor:h5_cursor + x.shape[0], :, :, :, :] = x
        h5_test_y[h5_cursor:h5_cursor + y.shape[0], :] = y

    # update cursor
    h5_cursor += len(x)

0 46 s23-4847
XX 0 (152, 2) .. (152, 5, 224, 224, 3)
1 46 s43-5211
XX 152 (217, 2) .. (217, 5, 224, 224, 3)
2 46 s2-1133
XX 369 (152, 2) .. (152, 5, 224, 224, 3)
3 46 s21-919
XX 521 (143, 2) .. (143, 5, 224, 224, 3)
4 46 s20-842
XX 664 (327, 2) .. (327, 5, 224, 224, 3)
5 46 s37-3930
XX 991 (115, 2) .. (115, 5, 224, 224, 3)
6 46 s5-1102
XX 1106 (299, 2) .. (299, 5, 224, 224, 3)
7 46 s19-672
XX 1405 (162, 2) .. (162, 5, 224, 224, 3)
8 46 s26-8164
XX 1567 (262, 2) .. (262, 5, 224, 224, 3)
9 46 s41-4712
XX 1829 (364, 2) .. (364, 5, 224, 224, 3)
10 46 s18-630
XX 2193 (346, 2) .. (346, 5, 224, 224, 3)
11 46 s25-5886
XX 2539 (189, 2) .. (189, 5, 224, 224, 3)
12 46 s35-3664
XX 2728 (143, 2) .. (143, 5, 224, 224, 3)
13 46 s33-3405
XX 2871 (309, 2) .. (309, 5, 224, 224, 3)
14 46 s45-6301
XX 3180 (143, 2) .. (143, 5, 224, 224, 3)
15 46 s16-0
XX 3323 (125, 2) .. (125, 5, 224, 224, 3)
16 46 s39-4336
XX 3448 (235, 2) .. (235, 5, 224, 224, 3)
17 46 s29-316
XX 3683 (401, 2) .. (401, 5, 224, 224, 3)
18

TypeError: Can't broadcast (1403, 5, 224, 224, 3) -> (427, 5, 224, 224, 3)

In [122]:
h5_cursor

10192

In [121]:
h5_shape_train_x

(10619, 5, 224, 224, 3)

In [123]:
h5_shape_train_x[0] - h5_cursor

427

In [103]:
# close h5 files
f_train.close()
f_valid.close()
f_test.close()

In [104]:
f.close()

## h5

In [ ]:
data_to_write = np.random.random(size=(100,20)) # or some such

In [27]:
data_to_write.shape

(100, 20)

In [48]:
data_to_write[[3,5],:]

array([[0.87288674, 0.0479354 , 0.21496869, 0.52457312, 0.36355362,
        0.5738816 , 0.51950428, 0.12935186, 0.65829541, 0.7267599 ,
        0.85523122, 0.60086587, 0.87617385, 0.6988311 , 0.47135551,
        0.13803998, 0.68947727, 0.83573227, 0.61033383, 0.23938601],
       [0.53659324, 0.88290034, 0.97825525, 0.38554264, 0.25060313,
        0.56811374, 0.713326  , 0.05882883, 0.33015174, 0.55246261,
        0.48120046, 0.40404853, 0.58548799, 0.73555519, 0.81325294,
        0.60537432, 0.19730834, 0.87475184, 0.55161567, 0.59570509]])

In [ ]:
with h5py.File('name-of-file.h5', 'w') as hf:
    hf.create_dataset("name-of-dataset",  data=data_to_write)

In [46]:
with h5py.File('name-of-file.h5', 'r') as hf:
    data = hf['name-of-dataset'][[3,5],:]
    print(data)
    print(type(data))
    print(data.shape)

[[0.87288674 0.0479354  0.21496869 0.52457312 0.36355362 0.5738816
  0.51950428 0.12935186 0.65829541 0.7267599  0.85523122 0.60086587
  0.87617385 0.6988311  0.47135551 0.13803998 0.68947727 0.83573227
  0.61033383 0.23938601]
 [0.53659324 0.88290034 0.97825525 0.38554264 0.25060313 0.56811374
  0.713326   0.05882883 0.33015174 0.55246261 0.48120046 0.40404853
  0.58548799 0.73555519 0.81325294 0.60537432 0.19730834 0.87475184
  0.55161567 0.59570509]]
<class 'numpy.ndarray'>
(2, 20)


## fit with no generator

In [22]:
# train model
fit_history = train(architecture.model, data, path_model = pwd+'models/', learning_rate = 0.001, epochs = 10)

Train on 10775 samples, validate on 1380 samples
Epoch 1/10
10775/10775 [==============================] - 118s 11ms/step - loss: 1.4871 - acc: 0.6240 - val_loss: 0.5746 - val_acc: 0.7812

Epoch 00001: val_acc improved from -inf to 0.78116, saving model to /mnt/seals/models/model.h5
Epoch 2/10
10775/10775 [==============================] - 111s 10ms/step - loss: 0.5598 - acc: 0.7252 - val_loss: 0.4317 - val_acc: 0.8449

Epoch 00002: val_acc improved from 0.78116 to 0.84493, saving model to /mnt/seals/models/model.h5
Epoch 3/10
10775/10775 [==============================] - 111s 10ms/step - loss: 0.4653 - acc: 0.7873 - val_loss: 0.3968 - val_acc: 0.8601

Epoch 00003: val_acc improved from 0.84493 to 0.86014, saving model to /mnt/seals/models/model.h5
Epoch 4/10
10775/10775 [==============================] - 111s 10ms/step - loss: 0.4131 - acc: 0.8214 - val_loss: 0.3566 - val_acc: 0.8696

Epoch 00004: val_acc improved from 0.86014 to 0.86957, saving model to /mnt/seals/models/model.h5
Ep

# Build cache

In [18]:
# if __name__ == "__main__":
#     # build feature cache in advance by running python3 data.py
#     for pretrained_model_name in pretrained_model_names:
#         for pooling in poolings:
#             data = Data(sequence_length=1, 
#                         return_CNN_features=True,
#                         pretrained_model_name = pretrained_model_name,
#                         pooling=pooling)

In [19]:
# fix penguin lables data
# labels = get_labels()
# labels = labels[labels['video'] != "20161014_no8_3"]
# labels = labels[labels['video'] != "20161014_no8_4"]
# labels.loc[labels['video'] == '20160930_no8_1_2','video']='20160930_no8_2'
# labels.to_csv(pwd + '/data/labels.csv',index=False)